In [1]:
import glob
import pandas as pd

In [2]:
dfs = dict()
for f in glob.glob('*.xls'):
    print(f)
    # Get year
    año = pd.read_excel(f, nrows=1)
    año = int(año.iloc[0,0][-4:])
    # Read file
    df = pd.read_excel(f, skiprows=6)
    df = pd.read_excel(f, dtype='str', skiprows=6)
    # Columns
    col_names = {0: 'Clave', 1:'Entidad', 2:'Edad', 3:'Sexo'}
    df.columns = [col_names.get(i, col) for i, col in enumerate(df.columns)]
    # Remove totals
    df = df.drop('Total', axis=1)
    df = df[~(df[col_names.values()]=='Total').any(axis=1)]
    df = df.drop(df.tail(1).index).dropna(how='all')
    # Melt
    df = pd.melt(df, id_vars = col_names.values(), value_vars=df.columns[4:],
                  var_name='Diagnostico', value_name='Frecuencia')
    df['Frecuencia'] = df['Frecuencia'].fillna('0')
    # Add year
    df.insert(0, 'Año', año)
    df
    # save
    print(año, df.shape)
    dfs[año] = df
    
df = pd.concat(dfs, ignore_index=True)

INEGI_exporta_2_10_2024_11_29_26.xls
2014 (137160, 7)
INEGI_exporta_2_10_2024_11_12_41.xls
2019 (139416, 7)
INEGI_exporta_2_10_2024_11_9_27.xls
2021 (150650, 7)
INEGI_exporta_2_10_2024_10_53_59.xls
2023 (140184, 7)
INEGI_exporta_2_10_2024_11_8_6.xls
2022 (150350, 7)
INEGI_exporta_2_10_2024_11_33_18.xls
2012 (137664, 7)
INEGI_exporta_2_10_2024_11_32_12.xls
2013 (138288, 7)
INEGI_exporta_2_10_2024_11_37_42.xls
2011 (139536, 7)
INEGI_exporta_2_10_2024_11_38_46.xls
2010 (138024, 7)
INEGI_exporta_2_10_2024_11_14_19.xls
2018 (141576, 7)
INEGI_exporta_2_10_2024_11_15_47.xls
2017 (140880, 7)
INEGI_exporta_2_10_2024_11_12_9.xls
2020 (150275, 7)
INEGI_exporta_2_10_2024_11_16_25.xls
2016 (144150, 7)
INEGI_exporta_2_10_2024_11_27_33.xls
2015 (138960, 7)


In [3]:
from pandas.api.types import CategoricalDtype

# Edad
cat_edad = list( df['Edad'].unique() )
cat_edad = CategoricalDtype(categories=cat_edad, ordered=True)
df['Edad'] = df['Edad'].astype(cat_edad)
# Sexo
df['Sexo'] = df[['Sexo']].astype('category')
# Frecuencia
df['Frecuencia'] = df['Frecuencia'].apply(lambda x: int(x.replace(',','')) )
# Clave
df['Clave'] = df['Clave'].apply(lambda x: str(x.replace(' ','')) )

df_edo = df[df['Clave'].str.len()<3]
df_edo.to_csv('data_raw/defuncion_nacion.csv', index=False)
display( df_edo.head() )

cve_valle = ['09002', '09003', '09004', '09005', '09006', '09007', '09008', '09009', '09010', '09011', '09012', 
             '09013', '09014', '09015', '09016', '09017', '15002', '15009', '15010', '15011', '15013', '15015', 
             '15016', '15017', '15020', '15022', '15023', '15024', '15025', '15028', '15029', '15030', '15031', 
             '15033', '15034', '15035', '15036', '15037', '15038', '15039', '15044', '15046', '15050', '15053', 
             '15057', '15058', '15059', '15060', '15061', '15065', '15068', '15069', '15070', '15075', '15081', 
             '15083', '15084', '15089', '15091', '15092', '15093', '15094', '15095', '15096', '15099', '15100', 
             '15103', '15104', '15108', '15109', '15112', '15120', '15121', '15122', '15125']
df_valle = df[df['Clave'].isin(cve_valle)]
df_edo.to_csv('data_raw/defuncion_valle.csv', index=False)
display( df_valle.head() )


,Año,Clave,Entidad,Edad,Sexo,Diagnostico,Frecuencia
0,2014,01,Aguascalientes,Menores de 1 año,Hombre,Ciertas enfermedades infecciosas y parasitarias,7
1,2014,01,Aguascalientes,Menores de 1 año,Mujer,Ciertas enfermedades infecciosas y parasitarias,2
2,2014,01,Aguascalientes,Menores de 1 año,No especificado,Ciertas enfermedades infecciosas y parasitarias,0
3,2014,01,Aguascalientes,1-4 años,Hombre,Ciertas enfermedades infecciosas y parasitarias,1
4,2014,01,Aguascalientes,1-4 años,Mujer,Ciertas enfermedades infecciosas y parasitarias,0


,Año,Clave,Entidad,Edad,Sexo,Diagnostico,Frecuencia
385,2014,09002,Azcapotzalco,Menores de 1 año,Hombre,Ciertas enfermedades infecciosas y parasitarias,9
386,2014,09002,Azcapotzalco,Menores de 1 año,Mujer,Ciertas enfermedades infecciosas y parasitarias,10
387,2014,09002,Azcapotzalco,Menores de 1 año,No especificado,Ciertas enfermedades infecciosas y parasitarias,0
388,2014,09002,Azcapotzalco,1-4 años,Hombre,Ciertas enfermedades infecciosas y parasitarias,2
389,2014,09002,Azcapotzalco,1-4 años,Mujer,Ciertas enfermedades infecciosas y parasitarias,5


In [4]:
import pandas as pd
from ydata_profiling import ProfileReport

prof = df_edo.profile_report()
prof.to_file(output_file='viz/defuncion_nacion_profile.html')

prof = df_valle.profile_report()
prof.to_file(output_file='viz/defuncion_valle_profile.html')

/home/esther/anaconda3/envs/SEMEFO_busqueda/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/esther/anaconda3/envs/SEMEFO_busqueda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
Export report to file: 100%|████████████████████| 1/1 [00:00<00:00, 1014.83it/s]
/home/esther/anaconda3/envs/SEMEFO_busqueda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h